In [2]:
import configparser
import mlflow, os
import pickle
import pandas as pd

In [11]:
config = configparser.ConfigParser()
config.read('config3.ini')
PATH_DATA_PREPROCESSOR = config.get('Paths', 'PATH_DATA_PREPROCESSOR')
PATH_PROCESSED_SCHEMA = config.get('Paths', 'PATH_PROCESSED_SCHEMA')
DIR_MLRUNS = config.get('Paths', 'DIR_MLRUNS')
DIR_DATA = config.get('Paths', 'DIR_DATA')
MODEL_NAME = config.get('Models', 'MODEL_NAME')
MODEL_VERSION = config.getint('Models', 'MODEL_VERSION')

In [4]:
data = pd.read_csv(f'{DIR_DATA}/Marketing.csv',encoding="utf-8")
data.head()

,age,sexe,taux,situationFamiliale,nbEnfantsAcharge,2emevoiture
0,21,F,1396,C�libataire,0,False
1,35,M,223,C�libataire,0,False
2,48,M,401,C�libataire,0,False
3,26,F,420,En Couple,3,True
4,80,M,530,En Couple,3,False


In [5]:
data.dtypes

age                    int64
sexe                  object
taux                   int64
situationFamiliale    object
nbEnfantsAcharge       int64
2emevoiture             bool
dtype: object

In [12]:
# Chargement du pre-traiteur
with open(PATH_DATA_PREPROCESSOR, 'rb') as file:
    data_preprocessor = pickle.load(file)

# Chargement du shema
with open(PATH_PROCESSED_SCHEMA, 'rb') as file:
    data_schema = pickle.load(file)

labels_map = data_schema["categories_map"]
labels_map

{0: 'Citadine',
 1: 'Haut de gamme',
 2: 'Familliale',
 3: 'Economique',
 4: 'Moyen'}

In [13]:
# Correction des données
data2 = data.copy()
data2.loc[data2.situationFamiliale == "C�libataire","situationFamiliale"] = "Célibataire"

In [14]:
X = data_preprocessor.transform(data2)

In [15]:
model_uri = f"models:/{MODEL_NAME}/{MODEL_VERSION}"
print(model_uri)

# Chargement du modèle
mlflow.set_tracking_uri("file:" + os.path.abspath(DIR_MLRUNS))
model = mlflow.pyfunc.load_model(model_uri=model_uri)

# Prediction
y_pred = model.predict(X)
y_pred

models:/vehicule_classifier_random_forest/20


array([3, 0, 0, 2, 1, 3, 4, 0, 3, 3, 2, 0, 0, 2, 0, 2, 2, 2, 0, 3])

In [16]:
data3 = data2.copy()
data3["vehicule_categorie_id"] = y_pred
data3["vehicule_categorie_label"] = data3.vehicule_categorie_id.map(labels_map)
data3

,age,sexe,taux,situationFamiliale,nbEnfantsAcharge,2emevoiture,vehicule_categorie_id,vehicule_categorie_label
0,21,F,1396,Célibataire,0,False,3,Economique
1,35,M,223,Célibataire,0,False,0,Citadine
2,48,M,401,Célibataire,0,False,0,Citadine
3,26,F,420,En Couple,3,True,2,Familliale
4,80,M,530,En Couple,3,False,1,Haut de gamme
5,27,F,153,En Couple,2,False,3,Economique
6,59,F,572,En Couple,2,False,4,Moyen
7,43,F,431,Célibataire,0,False,0,Citadine
8,64,M,559,Célibataire,0,False,3,Economique
9,22,M,154,En Couple,1,False,3,Economique


In [10]:
from model_wrapper import VehiculeClassificaionModel

model2 =  VehiculeClassificaionModel("config.ini")
result = model2.predict(data)
display(result)
final_data = model2.postprocess(result)
final_data

array([1, 0, 0, 2, 2, 3, 3, 1, 0, 3, 3, 0, 1, 3, 1, 2, 3, 2, 1, 1])

,age,sexe,taux,situationFamiliale,nbEnfantsAcharge,2emevoiture,vehicule_categorie_id,vehicule_categorie_label
0,21,F,1396,C�libataire,0,False,1,Citadine
1,35,M,223,C�libataire,0,False,0,Standard
2,48,M,401,C�libataire,0,False,0,Standard
3,26,F,420,En Couple,3,True,2,Luxe et sportive
4,80,M,530,En Couple,3,False,2,Luxe et sportive
5,27,F,153,En Couple,2,False,3,Familiale
6,59,F,572,En Couple,2,False,3,Familiale
7,43,F,431,C�libataire,0,False,1,Citadine
8,64,M,559,C�libataire,0,False,0,Standard
9,22,M,154,En Couple,1,False,3,Familiale
